![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Tool Loadout: Dynamic Tool Selection

## Learning Objectives (35 minutes)
By the end of this notebook, you will be able to:
1. **Understand** the concept of tool loadout and why it matters for agent performance
2. **Implement** semantic tool selection using vector similarity
3. **Apply** dynamic tool filtering based on user intent and context
4. **Optimize** agent performance by reducing tool confusion and token usage
5. **Design** tool recommendation systems for large tool inventories

## Prerequisites
- Completed Sections 1-4 of the Context Engineering course
- Understanding of vector embeddings and semantic search
- Familiarity with LangChain tools and function calling

---

## Introduction

**Tool Loadout** is the practice of selecting only the most relevant tools for a given task, rather than providing an agent with access to all available tools. The term comes from gaming, where players select specific combinations of weapons and equipment before a mission.

### Why Tool Loadout Matters

Research shows that agent performance degrades significantly when given too many tools:

- **DeepSeek-v3**: Performance drops after 30 tools, fails completely with 100+ tools
- **Llama 3.1 8B**: Fails benchmarks with 46 tools, succeeds with only 19 tools
- **Context Confusion**: Too many similar tools create decision paralysis
- **Token Waste**: Unused tool descriptions consume valuable context space

### The Tool Loadout Solution

Instead of giving agents access to all tools, we:
1. **Analyze the user's request** to understand intent
2. **Select relevant tools** using semantic similarity
3. **Provide only the necessary tools** to the agent
4. **Optimize for both accuracy and efficiency**

## Environment Setup

Let's set up our environment for tool loadout experiments:

In [ ]:
# Environment setup
import os
import asyncio
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional
import json
from dataclasses import dataclass
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load environment variables
load_dotenv()

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"Redis URL: {REDIS_URL}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

In [ ]:
# Import required modules
try:
    from openai import OpenAI
    from redis_context_course.models import Course, StudentProfile
    from redis_context_course.course_manager import CourseManager
    
    # Initialize OpenAI client
    if OPENAI_API_KEY:
        openai_client = OpenAI(api_key=OPENAI_API_KEY)
        print("✅ OpenAI client initialized")
    else:
        openai_client = None
        print("⚠️  OpenAI client not available (API key not set)")
    
    print("✅ Core modules imported successfully")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure you've completed the setup from previous sections.")

## Tool Definition and Management

Let's start by defining a comprehensive set of tools that our agent might have access to:

In [ ]:
@dataclass
class ToolDefinition:
    """Represents a tool with its metadata for selection."""
    name: str
    description: str
    category: str
    parameters: Dict[str, Any]
    embedding: Optional[np.ndarray] = None
    usage_frequency: int = 0
    
    def to_openai_format(self) -> Dict[str, Any]:
        """Convert to OpenAI function calling format."""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters
            }
        }

# Define a comprehensive tool inventory
TOOL_INVENTORY = [
    # Course Management Tools
    ToolDefinition(
        name="search_courses",
        description="Search for courses using semantic similarity and filters. Use for finding courses by topic, difficulty, or format.",
        category="course_management",
        parameters={
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query for courses"},
                "limit": {"type": "integer", "description": "Maximum number of results"}
            },
            "required": ["query"]
        }
    ),
    ToolDefinition(
        name="get_course_details",
        description="Get detailed information about a specific course including prerequisites, schedule, and enrollment.",
        category="course_management",
        parameters={
            "type": "object",
            "properties": {
                "course_code": {"type": "string", "description": "Course code (e.g., CS101)"}
            },
            "required": ["course_code"]
        }
    ),
    ToolDefinition(
        name="check_prerequisites",
        description="Check if a student meets the prerequisites for a specific course.",
        category="course_management",
        parameters={
            "type": "object",
            "properties": {
                "course_code": {"type": "string", "description": "Course code to check"},
                "student_courses": {"type": "array", "items": {"type": "string"}, "description": "List of completed courses"}
            },
            "required": ["course_code", "student_courses"]
        }
    ),
    
    # Student Profile Tools
    ToolDefinition(
        name="get_student_profile",
        description="Retrieve comprehensive student profile including academic history, preferences, and goals.",
        category="student_management",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"}
            },
            "required": ["student_id"]
        }
    ),
    ToolDefinition(
        name="update_student_preferences",
        description="Update student preferences for course format, difficulty, or schedule.",
        category="student_management",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"},
                "preferences": {"type": "object", "description": "Preference updates"}
            },
            "required": ["student_id", "preferences"]
        }
    ),
    
    # Academic Planning Tools
    ToolDefinition(
        name="generate_degree_plan",
        description="Generate a comprehensive degree completion plan based on student's major and progress.",
        category="academic_planning",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"},
                "target_graduation": {"type": "string", "description": "Target graduation date"}
            },
            "required": ["student_id"]
        }
    ),
    ToolDefinition(
        name="check_graduation_requirements",
        description="Check progress toward graduation requirements for a specific major.",
        category="academic_planning",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"},
                "major": {"type": "string", "description": "Academic major"}
            },
            "required": ["student_id", "major"]
        }
    ),
    
    # Schedule Management Tools
    ToolDefinition(
        name="check_schedule_conflicts",
        description="Check for time conflicts between courses in a proposed schedule.",
        category="schedule_management",
        parameters={
            "type": "object",
            "properties": {
                "course_codes": {"type": "array", "items": {"type": "string"}, "description": "List of course codes"}
            },
            "required": ["course_codes"]
        }
    ),
    ToolDefinition(
        name="find_available_sections",
        description="Find available sections for a course that fit student's schedule preferences.",
        category="schedule_management",
        parameters={
            "type": "object",
            "properties": {
                "course_code": {"type": "string", "description": "Course code"},
                "time_preferences": {"type": "array", "items": {"type": "string"}, "description": "Preferred time slots"}
            },
            "required": ["course_code"]
        }
    ),
    
    # Financial Tools
    ToolDefinition(
        name="calculate_tuition_cost",
        description="Calculate total tuition cost for a set of courses.",
        category="financial",
        parameters={
            "type": "object",
            "properties": {
                "course_codes": {"type": "array", "items": {"type": "string"}, "description": "List of course codes"},
                "student_type": {"type": "string", "description": "Student type (undergraduate, graduate, etc.)"}
            },
            "required": ["course_codes"]
        }
    ),
    ToolDefinition(
        name="check_financial_aid",
        description="Check available financial aid options for a student.",
        category="financial",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"}
            },
            "required": ["student_id"]
        }
    ),
    
    # Career Services Tools
    ToolDefinition(
        name="find_career_paths",
        description="Find career paths and job opportunities related to a student's major and interests.",
        category="career_services",
        parameters={
            "type": "object",
            "properties": {
                "major": {"type": "string", "description": "Academic major"},
                "interests": {"type": "array", "items": {"type": "string"}, "description": "Student interests"}
            },
            "required": ["major"]
        }
    ),
    ToolDefinition(
        name="recommend_internships",
        description="Recommend internship opportunities based on student profile and career goals.",
        category="career_services",
        parameters={
            "type": "object",
            "properties": {
                "student_id": {"type": "string", "description": "Student identifier"},
                "career_goals": {"type": "array", "items": {"type": "string"}, "description": "Career goals"}
            },
            "required": ["student_id"]
        }
    )
]

print(f"📚 Tool Inventory: {len(TOOL_INVENTORY)} tools defined")
print("\n📋 Tool Categories:")
categories = {}
for tool in TOOL_INVENTORY:
    categories[tool.category] = categories.get(tool.category, 0) + 1

for category, count in categories.items():
    print(f"   • {category}: {count} tools")